## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [24]:
from TCGame_Env1 import TicTacToe #import your class from environment file
import collections
import numpy as np
import random
import pickle
import time
import copy
import matplotlib.pyplot as plt

In [25]:
env = TicTacToe()

tempState = [0,float('nan'),float('nan'),3,4,5,6,7,8]
for count in range(3):
    row_list = [value for index,value in enumerate(tempState) if index>=count*3 and index<3+count*3]
    print(row_list)

[0, nan, nan]
[3, 4, 5]
[6, 7, 8]


In [26]:
print(env.is_winning(tempState))

testList = [0,float('nan'),float('nan'),3,4,float('nan'),6,float('nan'),float('nan')]
print(env.step(testList, [7, 8]))

True
([0, nan, nan, 3, 4, nan, 6, 8, 2], -10, True)


In [27]:
random.choice([i for i in env.action_space(testList)[1]])

(8, 2)

In [28]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):
    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [29]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):
    valid_Actions = []
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [30]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=1.0

#### Epsilon-greedy strategy

In [45]:
# Defining epsilon-greedy policy.
def epsilon_greedy(state, episode):
    max_epsilon = 1.0
    min_epsilon = 0.001
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.0001*episode)
    z = np.random.random()  
    if z > epsilon:
        action = max(Q_dict[Q_state(state)],key=Q_dict[Q_state(state)].get)  
    else:
        action = random.choice([i for i in env.action_space(state)[0]])
    return action


#### Tracking the state-action pairs for checking convergence - write your code here

In [46]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)
States_track = collections.defaultdict(dict)
Rewards = collections.defaultdict(dict)

print(len(Q_dict))
print(len(States_track))
print(len(Rewards))

0
0
0


In [47]:
# Initialise states to be tracked
rewards_tracked = {('4-x-x-x-x-x-x-x-3',(1,1)):[],('x-9-x-x-x-4-x-x-x',(0,1)):[], ('x-x-x-x-x-x-x-x-x',(4, 9)): [], ('x-x-x-x-x-8-7-x-x',(1, 5)):[], ('4-x-x-x-7-x-x-6-3',(2, 1)):[]}
def initialise_tracking_states():
    sample_q_values = [('4-x-x-x-x-x-x-x-3',(1,1)),
                       ('x-9-x-x-x-4-x-x-x',(0,1)),           
                       ('x-x-x-x-x-x-x-x-x',(4,9)),
                       ('x-x-x-x-x-8-7-x-x',(1,5)),
                       ('4-x-x-x-7-x-x-6-3',(2,1))]
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []    

In [48]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [49]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[Q_state(state)]:
                States_track[state][action].append(Q_dict[Q_state(state)][action])

In [50]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [51]:
#Defining parameters for the experiment
EPISODES = 100000
LR = 0.4                 #learning rate
GAMMA = 0.91

threshold = 20       #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
policy_threshold = 30    #every these many episodes, the Q-dict will be updated

### Q-update loop ---write your code here

In [52]:
import time
start_time = time.time()

agent_win = 0
env_win = 0
tie = 0

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    time_step = 0
    reward = None
    total_reward = 0
    TERMINAL = False

    initial_state = copy.deepcopy(env.state)
    curr_state = copy.deepcopy(env.state)  
    add_to_dict(curr_state)

    while not(TERMINAL):        
        # Agent Selecting a Epsilon Greedy action
        curr_action = epsilon_greedy(curr_state, episode)
        next_state, reward, TERMINAL = env.step(curr_state, curr_action)
        add_to_dict(next_state)

        # UPDATE RULE
        max_next = max(Q_dict[Q_state(curr_state)],key=Q_dict[Q_state(curr_state)].get)   
        Q_dict[Q_state(curr_state)][curr_action] += LR * ((reward + (GAMMA*(Q_dict[Q_state(curr_state)][max_next]))) - Q_dict[Q_state(curr_state)][curr_action])

        curr_state = next_state
        
        total_reward += reward

        if TERMINAL:            
            # Tracking the count of games won by agent and environment
            if reward == -10:
                env_win = env_win +1   
            elif reward == 10:
                agent_win = agent_win +1
            elif reward == 0:
                tie = tie+1
            if (episode + 1) % (EPISODES*0.1) == 0:
                print(" %d games, Agent Wins : %.4f, Environment Wins : %.4f, Tie : %.4f"% (episode+1, agent_win /(episode + 1), env_win /(episode+1), tie /(episode + 1)))
                    
    #TRACKING Q-VALUES
    if (episode == threshold-1):        #at the 1999th episode
        initialise_tracking_states()
      
    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_tracking_states()
        save_obj(States_track,'States_tracked')   
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0: #every 30000th
        save_obj(Q_dict,'Policy')    

elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')   

save_obj(Q_dict,'Policy')

 10000 games, Agent Wins : 0.4998, Environment Wins : 0.5002, Tie : 0.0000
 20000 games, Agent Wins : 0.5361, Environment Wins : 0.4639, Tie : 0.0000
 30000 games, Agent Wins : 0.5708, Environment Wins : 0.4292, Tie : 0.0000
 40000 games, Agent Wins : 0.6022, Environment Wins : 0.3978, Tie : 0.0000
 50000 games, Agent Wins : 0.6225, Environment Wins : 0.3775, Tie : 0.0000
 60000 games, Agent Wins : 0.6347, Environment Wins : 0.3654, Tie : 0.0000
 70000 games, Agent Wins : 0.6481, Environment Wins : 0.3519, Tie : 0.0000
 80000 games, Agent Wins : 0.6632, Environment Wins : 0.3368, Tie : 0.0000
 90000 games, Agent Wins : 0.6811, Environment Wins : 0.3189, Tie : 0.0000
 100000 games, Agent Wins : 0.6999, Environment Wins : 0.3001, Tie : 0.0000


In [53]:
elapsed_time

930.0692617893219

In [54]:
# with open('Rewards.pkl', 'rb') as handle:
#     States_track = pickle.load(handle) 
with open('Policy.pkl', 'rb') as handle:
    Q_dict = pickle.load(handle)
with open('States_tracked.pkl', 'rb') as handle:
    States_track = pickle.load(handle)    

print(len(Rewards))    
print(len(Q_dict))
print(len(States_track))

0
101708
14


#### Check the Q-dictionary

In [41]:
for index, data in enumerate(Q_dict.keys()):
    if index < 10:
        for i in Q_dict.get(data):
            print(str(data)+' : '+str(i)+' : '+str(Q_dict.get(data)[i]))
            # print(str(Q_dict.get(data)[i]))

x-x-x-x-x-x-x-x-x : (0, 1) : -11.106672874138692
x-x-x-x-x-x-x-x-x : (0, 3) : -11.10680434553889
x-x-x-x-x-x-x-x-x : (0, 5) : -11.106683170969895
x-x-x-x-x-x-x-x-x : (0, 7) : -11.10669893079669
x-x-x-x-x-x-x-x-x : (0, 9) : -11.106686206392656
x-x-x-x-x-x-x-x-x : (1, 1) : -11.106776028956501
x-x-x-x-x-x-x-x-x : (1, 3) : -11.106703563905779
x-x-x-x-x-x-x-x-x : (1, 5) : -11.106676595795129
x-x-x-x-x-x-x-x-x : (1, 7) : -11.10667222073657
x-x-x-x-x-x-x-x-x : (1, 9) : -11.106669616603554
x-x-x-x-x-x-x-x-x : (2, 1) : -11.106701573882773
x-x-x-x-x-x-x-x-x : (2, 3) : -11.106754127899992
x-x-x-x-x-x-x-x-x : (2, 5) : -11.10680390333214
x-x-x-x-x-x-x-x-x : (2, 7) : -11.106735058212507
x-x-x-x-x-x-x-x-x : (2, 9) : -11.106730423519183
x-x-x-x-x-x-x-x-x : (3, 1) : -11.106785675469013
x-x-x-x-x-x-x-x-x : (3, 3) : -11.106684072941
x-x-x-x-x-x-x-x-x : (3, 5) : -11.106765610669068
x-x-x-x-x-x-x-x-x : (3, 7) : -11.10669836251737
x-x-x-x-x-x-x-x-x : (3, 9) : -11.106788850215478
x-x-x-x-x-x-x-x-x : (4, 1) :

In [42]:
States_track

defaultdict(<class 'dict'>, {'x-3-x-x-x-6-x-x-x': {(0, 1): []}, 'x-1-x-x-x-x-8-x-x': {(2, 9): []}, 'x-x-x-x-6-x-x-x-5': {(2, 7): []}, 'x-x-x-x-9-x-6-x-x': {(1, 7): []}, 'x-5-x-2-x-x-4-7-x': {(0, 9): []}, '9-x-5-x-x-x-8-x-4': {(1, 3): []}, '2-7-x-x-6-x-x-3-x': {(8, 5): []}, '9-x-x-x-x-2-x-x-x': {(2, 5): []}, 'x-x-7-x-x-x-x-x-2': {(1, 5): []}, '5-x-x-x-x-6-x-x-x': {(4, 9): []}, '4-x-x-6-x-x-3-1-x': {(8, 5): []}, '5-x-8-x-x-6-3-x-x': {(3, 1): []}, 'x-6-5-x-2-x-x-3-x': {(0, 7): []}, '7-x-5-x-2-x-x-x-6': {(1, 3): []}})

In [43]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated

IndexError: list index out of range

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [44]:
# Write the code for plotting the graphs for state-action pairs tracked

IndexError: list index out of range

### Epsilon - decay check

In [20]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.001*i))

In [21]:
plt.plot(time, epsilon)
plt.show()

### Checking the states tracked for Q-values convergence

In [73]:

plt.figure(0, figsize=(16,7))
plt.subplot(241)
t1=States_track['x-x-8-7-x-5-6-3-x'][(0,9)]
plt.title("(s,a)=('x-x-8-7-x-5-6-3-x',(0,9))")
plt.plot(np.asarray(range(0, len(t1))),np.asarray(t1))

plt.subplot(242)
t2=States_track['6-x-9-x-x-3-2-x-7'][(1, 5)]
plt.title("(s,a)=('6-x-9-x-x-3-2-x-7',(1, 5))")
plt.plot(np.asarray(range(0, len(t2))),np.asarray(t2))

plt.subplot(243)
t3=States_track['x-x-2-6-x-5-9-x-x'][(0, 7)]
plt.title("(s,a)=('x-x-2-6-x-5-9-x-x',(0, 7))")
plt.plot(np.asarray(range(0, len(t3))),np.asarray(t3))

plt.subplot(244)
t4=States_track['x-x-x-7-x-4-x-x-x'][(1, 1)]
plt.title("(s,a)=('x-x-x-7-x-4-x-x-x',(1, 1))")
plt.plot(np.asarray(range(0, len(t4))),np.asarray(t4))

plt.show()

KeyError: (0, 9)